
# House Prices - Advanced Regression Techniques

In this notebook we will use a dataset provided by **Kaggle** for the competition [House Prices - Advanced Regression Techniques](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques).  

First we will explore the data and do some basic EDA, and then we will implement both a Random Forest and a neural network model.

Before we start we need to import the resources (libraries, modules, etc.) that we will be using.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import root_mean_squared_error

import tensorflow as tf
import tensorflow_decision_forests as tfdf
from tensorflow.keras.layers import Input, Dense, Concatenate, Normalization, StringLookup, CategoryEncoding
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import RootMeanSquaredError

As the data provided has many features, we will set up **Pandas** to display all columns and rows, making it easier to explore the data.

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Initial exploration

The data provided by Kaggle has already been separated into train and test datasets, each in a different CSV file.  
Both datasets have the same format, the only difference is our target variable **SalePrice** is missing in the test set.
Along with the data, a file called data_description.txt is provided which contains a description of each column (and of their values in the categorical ones).

We will start by loading the training data and doing some basic exploration to understand what we are dealing with.

In [ ]:
trainrawdata_path = '../data/raw/train.csv' # Path to the training dataset
traindf = pd.read_csv(trainrawdata_path)

In [ ]:
traindf.info()

We can see that our training dataset has 1460 datapoints, with 79 features (81 columns but one is **Id** and another is the target variable **SalePrice**).  
There seems to be missing data in some features, which we will explore later.

In [ ]:
traindf.describe()

In [ ]:
traindf.head(10)

After this first look we see that we have a mixed set of data, some columns are quantitative (numerical) like our target variable **SalePrice** or **LotArea** while many others are qualitative (categorical).  
Some of the categorical features are nominal but there are also ordinal variables like **OverallCond** which rates from 1 to 10 the overall condition of the house.

Lets have an initial look at our target variable **Saleprice**:

In [ ]:
traindf['SalePrice'].describe()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(traindf['SalePrice'], kde=True)
plt.title('Distribution of SalePrice')
plt.xlabel('SalePrice')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(np.log1p(traindf['SalePrice']), kde=True)
plt.title('Log-Transformed SalePrice')
plt.xlabel('Log(SalePrice)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(8, 1))
sns.boxplot(x=traindf['SalePrice'])
plt.title('Boxplot of SalePrice')
plt.show()

After an initial look at our target variable **SalePrice** we can see a right-skewed distribution.  
This is to be expected as most houses will sell around the same price with the more expensive ones being fewer in number.

## 2. Missing Values Analysis & Handling

### 2.1 Missing values on training dataset

Now lets look at which features have missing values and how many each have.

In [ ]:
traindf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]


As per the data_description.txt file, many of these features use NA as intented value, meaning "None".  
But there are other variables where that is not the case, like **GarageYrBlt** or **MasVnrArea**.  

As there are not that many features, lets explore them manually and deal with them accordingly.

**PoolQC** is a categorical variable which describes the quality of the pool, NA is not one of the categories but given that there is no category for "No pool", those 1453 missing values should mean that those 1453 houses have no pool.  

We can confirm this looking at how many houses have 0 pool area and checking if those entries are the same as the ones with **PoolQC** missing values.

In [ ]:
(traindf['PoolArea'] == 0).sum()

In [ ]:
traindf[(traindf['PoolArea'] == 0) & (traindf['PoolQC'].isnull())].shape

Now lets transform empty values into a string "None" to avoid issues with missing values downstream.

In [ ]:
traindf['PoolQC'] = traindf['PoolQC'].fillna('None')
traindf['PoolQC'].unique()

**MiscFeature** is also a categorical variable with NA as intended value for None, lets compare it with **MiscVal** which represents the value of said feature.

In [ ]:
(traindf['MiscVal'] == 0).sum()

In [ ]:
traindf[(traindf['MiscVal'] == 0) & (traindf['MiscFeature'].isnull())].shape

There seems to be 2 instances of **MiscVal** 0 more than the number of entries with **MiscFeature** as NA.  

In [ ]:
traindf[(traindf['MiscVal'] == 0) & (traindf['MiscFeature'].notna())]

As there are only 2 entries, we will drop them and replace the NA values of the rest of **MiscFeature** with "None".

In [ ]:
traindf = traindf.drop(index=traindf[(traindf['MiscVal'] == 0) & (traindf['MiscFeature'].notna())].index)

In [ ]:
traindf['MiscFeature'] = traindf['MiscFeature'].fillna('None')
traindf['MiscFeature'].unique()

Both **Alley** and **Fence** also use NA as None. This time there is no information to crosscheck, so we will assume all NA values are correct.

In [ ]:
traindf['Alley'] = traindf['Alley'].fillna('None')
traindf['Alley'].unique()

In [ ]:
traindf['Fence'] = traindf['Fence'].fillna('None')
traindf['Fence'].unique()

**MasVnrType** is a categorical variable that describes the type of masonry veneer, and **MasVnrArea** is a numerical variable that measures its area in square feet.  
For the type there is a None category but with None instead of NA. Lets check its values:

In [ ]:
(traindf['MasVnrArea'] == 0).sum()

In [ ]:
traindf['MasVnrArea'].isnull().sum()

In [ ]:
traindf['MasVnrType'].isnull().sum()

There seems to be some type of inconsistency here, as we have 870 missing values on **MasVnrType**, 8 missing values on **MasVnrArea**, and 859 values of 0 area.  
We should first check which rows have unexpected values.

In [ ]:
traindf[(traindf['MasVnrArea'] > 0) & (traindf['MasVnrType'].isnull())]

In [ ]:
traindf[(traindf['MasVnrArea'].isnull())]

In [ ]:
traindf[(traindf['MasVnrArea'] == 0) & ~(traindf['MasVnrType'].isnull())]

We can see 4 different cases here:  
> 8 entries where both type and area are NA  
> 2 entries where type is NA but where the area is 1.0, which would not make sense as that area value is too small  
> 3 entries where type is NA but where the area has a reasonable value  
> 2 entries where there is a valid type but the area is 0  

We are going to drop the entries with both values missing, and the two with 1.0 as area, because they are only 10 entries (<1% of the total).  

As for the other two cases, we are going to replace the missing values with the mode of the type from its neighborhood, and with the median of the neighborhood of the area.

In [ ]:
traindf = traindf.drop(index=traindf[(traindf['MasVnrType'].isnull()) & (traindf['MasVnrArea'].isnull())].index)
traindf = traindf.drop(index=traindf[(traindf['MasVnrArea'] == 1.0)].index)

In [ ]:
# Create boolean mask for those rows where MasVnrType is NaN and MasVnrArea is not 0
mask1 = traindf['MasVnrType'].isna() & (traindf['MasVnrArea'] != 0)

# Create boolean mask for those rows where MasVnrType has a valid value and MasVnrArea is 0
mask2 = ~traindf['MasVnrType'].isna() & (traindf['MasVnrArea'] == 0)

# Group by Neighborhood and get the mode of MasVnrType by Neighborhood and the median of MasVnrArea.
MasVnrType_mode_Neighborhood = (traindf.groupby('Neighborhood')['MasVnrType'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
MasVnrArea_median_Neighborhood = traindf.groupby('Neighborhood')['MasVnrArea'].median()

# Map the mode values to the original DataFrame
traindf.loc[mask1, 'MasVnrType'] = traindf.loc[mask1, 'Neighborhood'].map(MasVnrType_mode_Neighborhood)
traindf.loc[mask2, 'MasVnrArea'] = traindf.loc[mask2, 'Neighborhood'].map(MasVnrArea_median_Neighborhood)

Lets check whether all the missing values remaining matches the number of properties without masonry veneer.

In [ ]:
print(traindf['MasVnrType'].isnull().sum())
print((traindf['MasVnrArea'] == 0).sum())
traindf[(traindf['MasVnrArea'] == 0) & ~(traindf['MasVnrType'].isnull())]

We can see there is still an entry with valid veneer type but area 0, this means that the median of that neighborhood is 0.  
As it is only one entry, our safest approach is to just drop this one entry.

In [ ]:
traindf = traindf.drop(index=traindf[(traindf['MasVnrArea'] == 0) & ~(traindf['MasVnrType'].isnull())].index)

Now we will replace the NA values in type by 'None'.

In [ ]:
traindf['MasVnrType'] = traindf['MasVnrType'].fillna('None')
traindf['MasVnrType'].unique()

**FireplaceQu** has NA as None, and the amount should match the amount of 0 **Fireplaces**.  
If so, we will just replace those NA with "None".

In [ ]:
(traindf['FireplaceQu'].isnull()).sum()

In [ ]:
(traindf['Fireplaces'] == 0).sum()

In [ ]:
traindf[(traindf['Fireplaces'] == 0) & (traindf['FireplaceQu'].isnull())].shape

In [ ]:
traindf['FireplaceQu'] = traindf['FireplaceQu'].fillna('None')
traindf['FireplaceQu'].unique()

**LotFrontage** shows the linear feet of street connected to the house.  
As there is a big number of missing values (~17%), dropping them would not be reasonable.  
Instead, we will replace those values by the median of the neighborhood.

In [ ]:
# Create boolean mask for those rows where LotFrontage is NA.
mask = traindf['LotFrontage'].isna()

# Group by Neighborhood and get the mode of LotFrontage by Neighborhood
LotFrontage_median_Neighborhood = traindf.groupby('Neighborhood')['LotFrontage'].median()

# Map the mode values to the original DataFrame
traindf.loc[mask, 'LotFrontage'] = traindf.loc[mask, 'Neighborhood'].map(LotFrontage_median_Neighborhood)

In [ ]:
(traindf['LotFrontage'].isnull()).sum()

Now we will check the garage related variables.
We have 81 missing values on **GarageQual, GarageType, GarageFinish, GarageYrBlt, GarageExposure**.  
All those are categorical and have NA as legitimate value for "no garage", except **GarageYrBlt** which is numerical (year the garage was built).  

Besides those, we can see two more variables related to the garage, **GarageArea** and **GarageCars** which are numerical variables.

Now we should check that those 81 missing values on each feature, they all match 81 unique entries, that at the same time should have all of them 0 in both Area and Cars.

In [ ]:
(traindf['GarageCars'] == 0).sum()

In [ ]:
(traindf['GarageArea'] == 0).sum()

In [ ]:
traindf[(traindf['GarageArea'] == 0) & (traindf['GarageCars'] == 0) & (traindf['GarageQual'].isnull()) 
        & (traindf['GarageType'].isnull()) & (traindf['GarageFinish'].isnull()) 
        & (traindf['GarageCond'].isnull()) & (traindf['GarageYrBlt'].isnull())].shape

After checking this we can safely replace NA values on the categorical variables with "None", and for **GarageYrBlt** we will replace with -1 as a placeholder to indicate there is no garage.

In [ ]:
for var in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    traindf[var] = traindf[var].fillna('None')
    print(f"Unique values in {var}: {traindf[var].unique()}")
traindf['GarageYrBlt'] = traindf['GarageYrBlt'].fillna(-1)
(traindf['GarageYrBlt'] == -1).sum()

As far as basement related variables, we have 11 in total:
> **BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2**, these 5 categorical features use NA as value for "no basement", and those are the ones that show missing values.  
> **BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF**, these 4 numerical features describe the area in square feet of the different sections of the basement, they have no missing values.  
> **BsmtFullBath, BsmtHalfBath**, these 2 numerical features describe the amount of full and half bathrooms that there are in the basement, they have no missing values.  

We expect to see that those entries with missing values in all 5 categorical features, should have 0 as value in all the numerical variables.  
First we will check that and replace the missing values with "None" and then we can focus in the rest of the missing values.

In [ ]:
BsmtCatCols = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
mask = traindf[BsmtCatCols].isnull().all(axis=1)

print("Basement 1 area: ", 
      traindf[mask]["BsmtFinSF1"].value_counts())

print("Basement 2 area: ", 
      traindf[mask]["BsmtFinSF2"].value_counts())

print("Unfinished basement area: ", 
      traindf[mask]["BsmtUnfSF"].value_counts())

print("Basement total area: ", 
      traindf[mask]["TotalBsmtSF"].value_counts())

print("Basement full bathrooms: ", 
      traindf[mask]["BsmtFullBath"].value_counts())

print("Basement half bathrooms: ", 
      traindf[mask]["BsmtHalfBath"].value_counts())

With that we can safely replace in those 37 entries the missing value with "None".

In [ ]:
traindf.loc[mask, BsmtCatCols] = traindf.loc[mask, BsmtCatCols].fillna('None')

Lets check what missing values remain.

In [ ]:
traindf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

In [ ]:
traindf[traindf["BsmtExposure"].isnull() | traindf["BsmtFinType2"].isnull()]

We can see that there is one entry with **BsmtFinType2** empty even though there is surface area and the other values make sense, and another entry where **BsmtExposure**'s value is missing even though the rest of the values indicate that there is an unfinished basement. Both cases seem to be missing information, not empty on purpose.  

Given that it is only 2 entries, we will drop them.

In [ ]:
traindf = traindf.drop(index=traindf[traindf["BsmtExposure"].isnull() | traindf["BsmtFinType2"].isnull()].index)

As for the entry with the **Electrical** feature missing, we will drop it too.

In [ ]:
traindf = traindf.drop(index=traindf[traindf["Electrical"].isnull()].index)

Before we proceed lets check all missing values have been dealt with:

In [ ]:
traindf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

An empty list means there are no more missing values in our data.

In [ ]:
traindf.shape

We have lost in total 16 rows (~1%), but now we have a clean dataset that will not give us problems when we implement any models that cannot deal with missing values natively.

Lets save the cleaned dataset into a CSV file before proceeding with the next steps.

In [ ]:
train_clean_output_path = '../data/processed/train_clean.csv'  # Output file path
traindf.to_csv(train_clean_output_path, index=False)

### 2.2 Missing values on test dataset

Now we will repeat the same operations with the test dataset provided, but without dropping any rows as we will need to make a prediction from all of them.  

We will start by loading the test set.

In [ ]:
rawtestdata_path = '../data/raw/test.csv' # Path to the test dataset
testdf = pd.read_csv(rawtestdata_path)

In [ ]:
testdf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

We have more features with missing values in the test dataset than in the train set, but most seem to be one or two entries.  

As we cannot drop rows on the test set given that we need to make a prediction for all of them, lets first apply carefully the same transformations as with the training set and see what remains.

In [ ]:
# Replace NA with 'None' in every missing PoolQC that has PoolArea = 0
mask = (testdf["PoolArea"] == 0) & (testdf["PoolQC"].isnull())
testdf.loc[mask, "PoolQC"] = 'None'

# Replace NA with 'None' in every missing MiscFeature that has MiscVal = 0
mask = (testdf["MiscVal"] == 0) & (testdf["MiscFeature"].isnull())
testdf.loc[mask, "MiscFeature"] = 'None'

# Replace NA with 'None' in every missing Alley and Fence
testdf['Alley'] = testdf['Alley'].fillna('None')
testdf['Fence'] = testdf['Fence'].fillna('None')

# For those with both MasVnrtype and MasVnrArea missing, we will first replace the area with the median of the neighborhood from the training set
mask = testdf['MasVnrType'].isna() & (testdf['MasVnrArea'].isna())
testdf.loc[mask, 'MasVnrArea'] = testdf.loc[mask, 'Neighborhood'].map(MasVnrArea_median_Neighborhood)
# Then replace the MasVnrType with the mode of the neighborhood from the training set on those rows with a valid MasVnrArea (>0)
mask = testdf['MasVnrType'].isna() & (testdf['MasVnrArea'] > 0)
testdf.loc[mask, 'MasVnrType'] = testdf.loc[mask, 'Neighborhood'].map(MasVnrType_mode_Neighborhood)
# And for those with MasVnrArea = 0 and MasVnrType missing, we will replace the type with 'None'
mask = testdf['MasVnrType'].isna() & (testdf['MasVnrArea'] == 0)
testdf.loc[mask, "MasVnrType"] = 'None'

# Replace NA with 'None' in every missing FireplaceQu that has Fireplaces = 0
mask = (testdf["Fireplaces"] == 0) & (testdf["FireplaceQu"].isnull())
testdf.loc[mask, "FireplaceQu"] = 'None'

# Replace NA with the median LotFrontage of the neighborhood from the training set
mask = testdf['LotFrontage'].isna()
testdf.loc[mask, 'LotFrontage'] = testdf.loc[mask, 'Neighborhood'].map(LotFrontage_median_Neighborhood)

# Replace NA with 'None' in every missing categorical Garage variables, with -1 in GarageYrBlt and with 0 in GarageArea and GarageCars
# But only for those entries where all Garage variables mean there is no garage
mask = (
    ((testdf['GarageArea'].isnull()) | (testdf['GarageArea'] == 0)) &
    ((testdf['GarageCars'].isnull()) | (testdf['GarageCars'] == 0)) &
    (testdf['GarageQual'].isnull()) &
    (testdf['GarageType'].isnull()) &
    (testdf['GarageFinish'].isnull()) &
    (testdf['GarageCond'].isnull()) &
    ((testdf['GarageYrBlt'].isnull()) | (testdf['GarageYrBlt'] == 0))
)

for var in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    testdf.loc[mask, var] = 'None'
testdf.loc[mask, 'GarageYrBlt'] = -1
testdf.loc[mask, 'GarageArea'] = 0
testdf.loc[mask, 'GarageCars'] = 0



# Replace NA with 'None' in every missing categorical Basement variables, and with 0 in the numerical ones
# But only for those entries where all Basement variables mean there is no basement
mask = (
    ((testdf['BsmtFinSF1'].isnull()) | (testdf['BsmtFinSF1'] == 0)) &
    ((testdf['BsmtFinSF2'].isnull()) | (testdf['BsmtFinSF2'] == 0)) &
    ((testdf['BsmtUnfSF'].isnull()) | (testdf['BsmtUnfSF'] == 0)) &
    ((testdf['TotalBsmtSF'].isnull()) | (testdf['TotalBsmtSF'] == 0)) &
    ((testdf['BsmtFullBath'].isnull()) | (testdf['BsmtFullBath'] == 0)) &
    ((testdf['BsmtHalfBath'].isnull()) | (testdf['BsmtHalfBath'] == 0)) &
    (testdf['BsmtQual'].isnull()) &
    (testdf['BsmtCond'].isnull()) &
    (testdf['BsmtExposure'].isnull()) &
    (testdf['BsmtFinType1'].isnull()) &
    (testdf['BsmtFinType2'].isnull())
)
for var in ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']:
    testdf.loc[mask, var] = 'None'
for var in ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']:
    testdf.loc[mask, var] = 0

Afther that transformation the missing values remaining are the following.

In [ ]:
testdf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

In [ ]:
testdf.isnull().any(axis=1).sum()

Most missing values have been cleaned, but there are a few remnants over 22 entries that we will deal with manually.  

Using the information provided in data_description.txt we will proceed with the following:

- **MSZoning** we will replace missing values with the mode by neighborhood from the training set
- **PoolQC** we will replace missing values with the mode by neighborhood from the training set (we previously already replaced those with 0 **PoolArea** by "None")
- **Utilities** we will replace missing values with the mode by neighborhood from the training set
- **Functional** we will replace missing values with "Typ" (From documentation: Assume typical unless deductions are warranted)
- **Exterior1st** we will replace missing values with the mode by neighborhood from the training set
- **Exterior2nd** we will replace missing values with the mode by neighborhood from the training set
- **KitchenQual** we will replace missing values with the mode by neighborhood from the training set
- **MiscFeature** we will replace missing values with "Other" category (we previously already replaced those with 0 **MiscVal** by "None)
- **SaleType** we will replace missing values with the mode by neighborhood from the training set

In [ ]:
mask = testdf['MSZoning'].isna()
MSZoning_mode_Neighborhood = (traindf.groupby('Neighborhood')['MSZoning'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'MSZoning'] = testdf.loc[mask, 'Neighborhood'].map(MSZoning_mode_Neighborhood)

mask = testdf['PoolQC'].isna()
PoolQC_mode_Neighborhood = (traindf.groupby('Neighborhood')['PoolQC'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'PoolQC'] = testdf.loc[mask, 'Neighborhood'].map(PoolQC_mode_Neighborhood)

mask = testdf['Utilities'].isna()
Utilities_mode_Neighborhood = (traindf.groupby('Neighborhood')['Utilities'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'Utilities'] = testdf.loc[mask, 'Neighborhood'].map(Utilities_mode_Neighborhood)

mask = testdf['Functional'].isna()
testdf.loc[mask, 'Functional'] = "Typ"

mask = testdf['Exterior1st'].isna()
Exterior1st_mode_Neighborhood = (traindf.groupby('Neighborhood')['Exterior1st'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'Exterior1st'] = testdf.loc[mask, 'Neighborhood'].map(Exterior1st_mode_Neighborhood)

mask = testdf['Exterior2nd'].isna()
Exterior2nd_mode_Neighborhood = (traindf.groupby('Neighborhood')['Exterior2nd'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'Exterior2nd'] = testdf.loc[mask, 'Neighborhood'].map(Exterior2nd_mode_Neighborhood)

mask = testdf['KitchenQual'].isna()
KitchenQual_mode_Neighborhood = (traindf.groupby('Neighborhood')['KitchenQual'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'KitchenQual'] = testdf.loc[mask, 'Neighborhood'].map(KitchenQual_mode_Neighborhood)

mask = testdf['MiscFeature'].isna()
testdf.loc[mask, 'MiscFeature'] = "Other"

mask = testdf['SaleType'].isna()
SaleType_mode_Neighborhood = (traindf.groupby('Neighborhood')['SaleType'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
testdf.loc[mask, 'SaleType'] = testdf.loc[mask, 'Neighborhood'].map(SaleType_mode_Neighborhood)

We should have left the Basement and Garage related variables.

In [ ]:
testdf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

In [ ]:
testdf.isnull().any(axis=1).sum()

They are only 9 entries, lets explore them:

In [ ]:
testdf[testdf.isnull().any(axis=1)]

We can see there is one entry where even though there is **GarageType** defined, the rest of the information related to the garage is missing, so we are going to assume there is no garage on the property and the type was a data error.  

The rest of the entries show enough information about basement and garage, so we will replace the missing parts using the mode by neighborhood on the categorical variables and the median on the numerical ones.

In [ ]:
# First lets deal with the row where we are assuming there is no garage
row_label = testdf[testdf["GarageCars"].isnull()].index[0]
# Now lets change the rest of the garage variables
for var in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    testdf.loc[row_label, var] = 'None'
testdf.loc[row_label, "GarageArea"] = 0.0
testdf.loc[row_label, "GarageCars"] = 0.0
testdf.loc[row_label, "GarageYrBlt"] = -1

# Now for the rest, lets replace the missing values of the categorical variables with the mode of the neighborhood from the training set
for var in ['GarageFinish', 'GarageQual', 'GarageCond', 'BsmtExposure', 'BsmtQual', 'BsmtCond']:
    mask = testdf[var].isna()
    mode = (traindf.groupby('Neighborhood')[var].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'None'))
    testdf.loc[mask, var] = testdf.loc[mask, 'Neighborhood'].map(mode)

# And for GarageYrBlt, the only numerical variable, we will replace it with the median of the neighborhood from the training set
GarageYrBlt_median_Neighborhood = traindf.groupby('Neighborhood')['GarageYrBlt'].median()
row_label = testdf[testdf["GarageYrBlt"].isnull()].index[0]
neighborhood = testdf.loc[row_label, 'Neighborhood']
testdf.loc[row_label, 'GarageYrBlt'] = GarageYrBlt_median_Neighborhood[neighborhood]

Let's do a final check to make sure all missing values have been dealt with:

In [ ]:
testdf.isnull().sum().sort_values(ascending=False)[lambda x: x > 0]

And now lets save the cleaned test dataset to a file.

In [ ]:
test_clean_output_path = '../data/processed/test_clean.csv'  # Output file path
testdf.to_csv(test_clean_output_path, index=False)

## 3. Feature engineering  

Given the information available, there are a few features that could provide useful information for our prediction task.  

We are going to create the following on both sets of data:
- **TotalBathrooms**: Including basement ones
- **TotalSF**: Both floors plus basement
- **FinishedSF**: Total livable area (excluding unfinished basement)
- **Has2ndFloor**: Yes/No
- **HasBasement**: Yes/No
- **HasGarage**: Yes/No
- **HasPool**: Yes/No
- **HouseAge**: Years from when it was build till sale
- **GarageAge**: Years from when it was build till sale
- **RemodelAge**: Years from last remodelation till sale
- **WasRemodeled**: Yes/No
- **QualityIndex**: Ratio expressing overall quality and condition
- **LotRatio**: Ratio expressing relative house to land size (area)

In [ ]:
traindf["TotalBathrooms"] = traindf["FullBath"] + (0.5 * traindf["HalfBath"]) + traindf["BsmtFullBath"] + (0.5 * traindf["BsmtHalfBath"])
testdf["TotalBathrooms"] = testdf["FullBath"] + (0.5 * testdf["HalfBath"]) + testdf["BsmtFullBath"] + (0.5 * testdf["BsmtHalfBath"])

traindf["TotalSF"] = traindf["TotalBsmtSF"] + traindf["1stFlrSF"] + traindf["2ndFlrSF"]
testdf["TotalSF"] = testdf["TotalBsmtSF"] + testdf["1stFlrSF"] + testdf["2ndFlrSF"]

traindf["FinishedSF"] = traindf["BsmtFinSF1"] + traindf["BsmtFinSF2"] + traindf["1stFlrSF"] + traindf["2ndFlrSF"]
testdf["FinishedSF"] = testdf["BsmtFinSF1"] + testdf["BsmtFinSF2"] + testdf["1stFlrSF"] + testdf["2ndFlrSF"]

traindf["Has2ndFloor"] = (traindf["2ndFlrSF"] > 0).astype(int)
testdf["Has2ndFloor"] = (testdf["2ndFlrSF"] > 0).astype(int)

traindf["HasBasement"] = (traindf["TotalBsmtSF"] > 0).astype(int)
testdf["HasBasement"] = (testdf["TotalBsmtSF"] > 0).astype(int)

traindf["HasGarage"] = (traindf["GarageArea"] > 0).astype(int)
testdf["HasGarage"] = (testdf["GarageArea"] > 0).astype(int)

traindf["HasPool"] = (traindf["PoolArea"] > 0).astype(int)
testdf["HasPool"] = (testdf["PoolArea"] > 0).astype(int)

traindf["HouseAge"] = traindf["YrSold"] - traindf["YearBuilt"]
testdf["HouseAge"] = testdf["YrSold"] - testdf["YearBuilt"]

traindf["GarageAge"] = traindf["YrSold"] - traindf["GarageYrBlt"]
testdf["GarageAge"] = testdf["YrSold"] - testdf["GarageYrBlt"]

traindf["RemodelAge"] = traindf["YrSold"] - traindf["YearRemodAdd"]
testdf["RemodelAge"] = testdf["YrSold"] - testdf["YearRemodAdd"]

traindf["WasRemodel"] = (traindf["YearRemodAdd"] != traindf["YearBuilt"]).astype(int)
testdf["WasRemodel"] = (testdf["YearRemodAdd"] != testdf["YearBuilt"]).astype(int)

traindf["QualityIndex"] = traindf["OverallQual"] * traindf["OverallCond"]
testdf["QualityIndex"] = testdf["OverallQual"] * testdf["OverallCond"]

traindf["LotRatio"] = traindf["GrLivArea"] / traindf["LotArea"]
testdf["LotRatio"] = testdf["GrLivArea"] / testdf["LotArea"]

Besides this, we will drop the **Id** column in both datasets, as it offers no useful information for this task.

In [ ]:
traindf.drop('Id', axis=1, inplace=True)
testdf.drop('Id', axis=1, inplace=True)

And lets save both engineered datasets.

In [ ]:
train_eng_output_path = '../data/processed/train_engineered.csv'  # Output file path
traindf.to_csv(train_eng_output_path, index=False)

test_eng_output_path = '../data/processed/test_engineered.csv'  # Output file path
testdf.to_csv(test_eng_output_path, index=False)

## 4. Preprocessing for modeling

We are going to build a series of different models, and the preprocessing necessary differs between them.  
As such, in this section we will only apply those transformations that are common to all of them.

### 4.1 Log-transform target variable

Let's transform our target variable **SalePrice** 

In [ ]:
traindf["SalePrice"] = np.log1p(traindf["SalePrice"]).astype(np.float32)

### 4.2 Sanitize variables

To avoid mixed data types problems, specially with the neural networks models, we will sanitize the data by making sure all data has the following format:  
- Numerical variables: float32
- Categorical variables: String

First we need separate into two lists, the categorical and numerical columns.

In [ ]:
cat_features = ['MSSubClass', 'MSZoning',  'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond',  'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', 'KitchenQual', 
       'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive',  'PoolQC', 'Fence', 'MiscFeature', 'MoSold',
       'SaleType', 'SaleCondition',  'Has2ndFloor', 'HasBasement', 'HasGarage',
       'HasPool', 'WasRemodel']

num_features = ['LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
        'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 
        'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
        'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
        'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
        'ScreenPorch', 'PoolArea', 'MiscVal',  'YrSold', 'TotalBathrooms',
        'TotalSF', 'FinishedSF', 'HouseAge', 'GarageAge', 'RemodelAge', 'QualityIndex', 'LotRatio']

And now we will loop over the features changing types where necessary.

In [ ]:
for cat in cat_features:
    traindf[cat] = traindf[cat].astype(str)
    testdf[cat] = testdf[cat].astype(str)

for num in num_features:
    traindf[num] = traindf[num].astype(np.float32)
    testdf[num] = testdf[num].astype(np.float32)

### 4.3 Data split for validation

Our training dataset is relatively small, and the test set has already been split beforehand.  

As such, we will split our training set with a 70/30 split for validation purposes.

In [ ]:
Y = traindf["SalePrice"]
X = traindf.drop("SalePrice", axis=1)

In [ ]:
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=33)
X_dev, X_hold, Y_dev, Y_hold = train_test_split(X_temp, Y_temp, test_size=1/3, random_state=33)

## 5. Baseline model

We will use Scikit-Learn to build a simple linear model.  

In this case we will use Ridge Regression instead of linear regression due to the high number of features (the regularization could help avoid overfitting).

First we will define the preprocessor that will encode the categorical variables using one-hot encoding.  
The numerical variables will not be scaled or normalized as it is not necessary for this model.  

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
        # numerical columns are passed through unchanged
        ('num', 'passthrough', num_features)
    ]
)

We will build the model, and fit it to our training data.  
Because this is just a baseline model used for comparison, we will leave the alpha parameter as its default value (1.0).

In [ ]:
Ridge_model = Pipeline([
    ("preprocess", preprocessor),
    ("model", Ridge(alpha=1.0))
])

In [ ]:
# Fit the model
Ridge_model.fit(X_train, Y_train)

Now lets evaluate the model on our dev set:

In [ ]:
# Predict on dev set
Y_pred = Ridge_model.predict(X_dev)

In [ ]:
Ridge_rmse_log = root_mean_squared_error(Y_dev, Y_pred)
Ridge_rmse = root_mean_squared_error(np.expm1(Y_dev), np.expm1(Y_pred))
print(f"Ridge RMSE (log scale): {Ridge_rmse_log:.4f}")
print(f"Ridge RMSE (original scale): {np.expm1(Ridge_rmse_log):.4f}")
print(f"Random Forest RMSE (dollars): {Ridge_rmse:.4f}")

And also on our holdout set for comparison purposes:

In [ ]:
# Predict on hold set
Y_pred = Ridge_model.predict(X_hold)

In [ ]:
Ridge_rmse_log = root_mean_squared_error(Y_hold, Y_pred)
Ridge_rmse = root_mean_squared_error(np.expm1(Y_hold), np.expm1(Y_pred))
print(f"Ridge RMSE (log scale): {Ridge_rmse_log:.4f}")
print(f"Ridge RMSE (original scale): {np.expm1(Ridge_rmse_log):.4f}")
print(f"Random Forest RMSE (dollars): {Ridge_rmse:.4f}")

## 6. Random Forest

We will use TensorFLow Decision Forests to build our Random Forest model.  

TFDF can handle internally the encoding of categorical features, and as such we will be using the cleaned and engineered datasets directly with no preprocessing.

In [ ]:
RF_model = tfdf.keras.RandomForestModel(
    task=tfdf.keras.Task.REGRESSION,        # Define the task (Regression)
    num_trees=300,                          # Number of trees in the forest
    max_depth=10,                           # Maximum depth of trees
    min_examples=5,                         # Minimum number of examples per leaf node
    categorical_algorithm="CART",           # Algorithm for handling categorical features
    compute_oob_variable_importances=True,  # Compute out-of-bag variable importances
)

In [ ]:
RF_model.fit(tfdf.keras.pd_dataframe_to_tf_dataset(
    pd.concat([X_train, Y_train], axis=1), 
    task=tfdf.keras.Task.REGRESSION, label="SalePrice"))

Lets check the out-of-bag performance metrics.

In [ ]:
RF_inspector = RF_model.make_inspector()
print(RF_inspector.evaluation())

And finally lets make the predictions with the dev set and evaluate how our model performed.

In [ ]:
Y_pred = RF_model.predict(tfdf.keras.pd_dataframe_to_tf_dataset(X_dev))

In [ ]:

RF_rmse_log = root_mean_squared_error(Y_dev, Y_pred)
RF_rmse = root_mean_squared_error(np.expm1(Y_dev), np.expm1(Y_pred))

print(f"Random Forest RMSE (log scale): {RF_rmse_log:.4f}")
print(f"Random Forest RMSE (original scale): {np.expm1(RF_rmse_log):.4f}")
print(f"Random Forest RMSE (dollars): {RF_rmse:.4f}")

There is a substantial improvement already over our baseline model:  
- Ridge Holdout RMSE (log scale): 0.1641
- RandomForest Holdout RMSE (log scale): 0.1383  

But let's finetune it and see if it can be improved further.

In [ ]:
def evaluate_rf_model(num_trees, max_depth, min_examples):
    RF_model = tfdf.keras.RandomForestModel(
        task=tfdf.keras.Task.REGRESSION,             # Define the task (Regression)
        num_trees=num_trees,                         # Number of trees in the forest
        max_depth=max_depth,                         # Maximum depth of trees
        min_examples=min_examples,                   # Minimum number of examples per leaf node
        categorical_algorithm="CART",
        compute_oob_variable_importances=False,      # Compute out-of-bag variable importances
    )

    # Fit the model to the training data
    RF_model.fit(tfdf.keras.pd_dataframe_to_tf_dataset(
        pd.concat([X_train, Y_train], axis=1), 
        task=tfdf.keras.Task.REGRESSION, label="SalePrice"), verbose=0)

    # Predict on dev set
    Y_pred = RF_model.predict(tfdf.keras.pd_dataframe_to_tf_dataset(X_dev))
    rmse = root_mean_squared_error(Y_dev, Y_pred)
    return rmse

In [ ]:
RF_tuning = []

for trees in [100, 300, 500]:
    for depth in [8, 10, 12]:
        for minex in [2, 5, 10]:
            # Evaluate model and get RMSE
            rmse = evaluate_rf_model(trees, depth, minex)
            
            # Append the results to the list
            RF_tuning.append({
                "num_trees": trees,
                "max_depth": depth,
                "min_examples": minex,
                "rmse": rmse
            })

RF_tuning_df = pd.DataFrame(RF_tuning)
RF_tuning_df = RF_tuning_df.sort_values(by="rmse")

In [ ]:
print(RF_tuning_df)

Lets now fit the best model and evaluate it against the hold out sample to check if overfit our model.

In [ ]:
RF_model = tfdf.keras.RandomForestModel(
    task=tfdf.keras.Task.REGRESSION,        # Define the task (Regression)
    num_trees=500,                          # Number of trees in the forest
    max_depth=12,                           # Maximum depth of trees
    min_examples=5,                         # Minimum number of examples per leaf node
    categorical_algorithm="CART",           # Algorithm for handling categorical features
    compute_oob_variable_importances=True,  # Compute out-of-bag variable importances
)

In [ ]:
RF_model.fit(tfdf.keras.pd_dataframe_to_tf_dataset(
    pd.concat([X_train, Y_train], axis=1), 
    task=tfdf.keras.Task.REGRESSION, label="SalePrice"))

In [ ]:
RF_inspector = RF_model.make_inspector()
print(RF_inspector.evaluation())

In [ ]:
Y_pred = RF_model.predict(tfdf.keras.pd_dataframe_to_tf_dataset(X_hold))

In [ ]:
RF_rmse_log = root_mean_squared_error(Y_hold, Y_pred)
RF_rmse = root_mean_squared_error(np.expm1(Y_hold), np.expm1(Y_pred))

print(f"Random Forest RMSE (log scale): {RF_rmse_log:.4f}")
print(f"Random Forest RMSE (original scale): {np.expm1(RF_rmse_log):.4f}")
print(f"Random Forest RMSE (dollars): {RF_rmse:.4f}")

Performance has not worsened (it got even better but that is probably just due to the small holdout sample).

## 7. MLP model

We will build an MLP model with preprocessing.  

First the model will normalize all numerical features, and it will encode the categorical ones.  

After preprocessing the input it will then use 2 fully connected layer, with 128 and 64 hidden units, and with 'Relu' as activation function.  

As output layer given that this is a regression task, it will have a fully connected layer with one unit and no activation.

In [ ]:
# Input layer
# This will create a dictionary which contains an input layer per feature, to help us preprocess them individually
inputs = {}
for name in num_features + cat_features:
    inputs[name] = Input(shape=(1,), name=name, dtype='float32' if name in num_features else 'string')

# Preprocessing for numerical
norm_layers = {}
for feature in num_features:
    norm = Normalization() # Create a Normalization layer
    norm.adapt(X_train[feature].values)  # Gets the mean and variance of the training data
    norm_layers[feature] = norm(inputs[feature]) # Apply normalization to the input feature

# Preprocessing for categorical
cat_layers = {}
for feature in cat_features:
    lookup = StringLookup(output_mode='int') # Layer to map strings to integers
    lookup.adapt(X_train[feature].values) # Learns all unique values (categories) of the feature to map them to integers
    
    vocab_size = lookup.vocabulary_size() # Number of unique values (categories) in the feature + 1 for the UNK token
    encoding = CategoryEncoding(output_mode='one_hot', num_tokens=vocab_size) # Convert the integer encoded feature to one-hot encoding
    
    int_encoded = lookup(inputs[feature]) # Apply the lookup layer to the features
    one_hot_encoded = encoding(int_encoded) # Apply the encoding layer to the integer encoded feature
    cat_layers[feature] = one_hot_encoded # Store the one-hot encoded tensor to the dict

# Combine all features
all_features = list(norm_layers.values()) + list(cat_layers.values())
x = Concatenate()(all_features)

# MLP
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(1)(x)

model = Model(inputs=inputs, outputs=output)

Now let's configure the model to use Adam as optimization algorithm, using Mean Squared Error as loss function and we will use RMSE as metric.

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=[RootMeanSquaredError()])

We need to transform our training data to the format TF expects (dict of column name - array of values).

In [ ]:
X_train_inputs = {
    name: X_train[name].astype(str).values if name in cat_features else X_train[name].values
    for name in num_features + cat_features
}

Anf finally lets train the model:

In [ ]:
model.fit(X_train_inputs, Y_train.values, epochs=20, batch_size=32, validation_split=0.2)

In [ ]:
dev_inputs = {name: X_dev[name].values for name in num_features + cat_features}
loss, rmse = model.evaluate(dev_inputs, Y_dev.values)
print("Test RMSE:", rmse)

(to be done next)
(enconding, scaling, normalizing)
## 8. TabNet model
## 9. Models comparison
## 10. Chosen model test prediction
and inverse transformation of predictions np.expm1()